Задание 2
1/1 point (graded)
Определите функцию check_server, которая принимает на вход переменную mode.

Если mode имеет значение "memory", программа должна вернуть строку "Memory is ok".
Если mode имеет значение "connection", программа должна вернуть строку "Connection is ok".
Для остальных случае программа должна выбросить исключение ValueError.

In [11]:
def check_server(mode):
    if mode == 'memory':  
        return 'Memory is ok'
    elif mode == 'connection':        
        return 'Connection is ok' 
    else:
        raise ValueError()

6. Обработка исключений

Для этой цели мы можем воспользоваться блоком отработки исключений — try-except. Он позволяет отрабатывать исключения, не прерывая хода программы. Без блока try-except в примере мы бы получили KeyError, здесь же мы печатаем строку Oops, key not found и продолжаем выполнение программы.

Если более формально, то мы пробуем выполнить часть программы, которая находится внутри try. Если эта часть кода выбрасывает исключение, то мы сразу переходим к выполнению части внутри except. Если же исключение не выбрасывается, то мы пропускаем часть внутри except. В любом случае мы выполняем программу дальше, как обычно.

In [12]:
try:
    print("Inside try block")
except:
    print("Inside except block")

Inside try block


7. Осознанная обработка исключений


Осознанная обработка исключений
Общая идея использования try-except такая:

вы знаете, что у вас может возникнуть исключение, и не можете его полностью избежать;
вы "отлавливаете" эти исключения, а в блок except помещаете логику, которая позволит вам из этой ситуации корректно выйти.
Когда мы вызываем raise, то делаем подачу: тут какие-то не такие данные, нужно исправить. Когда мы обрабатываем исключения через try-except, то принимаем: да, данные неидеальные, но я об этом подумал. Проблема в том, что с обычным try-except можно "отловить" не только те исключения, на которые мы рассчитывали. 

Посмотрим на примере: у нас есть программа, которая отрабатывает очень большую базу данных, и мы бы хотели, чтобы была возможность корректно прервать её с клавиатуры. Код мог бы выглядеть как-то так:

In [13]:
try:  
    # запускаем программу отработки базы данных  
    process(db)   
except:  
    # если с клавиатуры пришло прерывание, то корректно останавливаем процесс  
    gently_close(db)  

NameError: name 'gently_close' is not defined

При остановке с клавиатуры мы получаем исключение KeyInterrupt, так что мы идём в блок except и корректно закрываем базу данных. Проблема в том, что except "отловит" и другие исключения, например, ошибку отсутствующего ключа или таймаута, что усложнит для нас обнаружение и исправление этих ошибок. На этот случай try-except позволяет указывать, какое именно исключение мы отлавливаем.

In [14]:
try:  
    process(db)   
except KeyInterrupt:  # обратите внимание, что мы добавили имя исключения после except  
    gently_close(db)  

NameError: name 'KeyInterrupt' is not defined

In [15]:
dictionary = {}  
try:  
    dictionary["no_key"]  
except KeyError:  
    print("Oops, key not found")  
# => Oops, key not found  
  
  
dictionary = {}  
try:  
    dictionary["no_key"]  
except IOError:  
    print("Oops, key not found")  

Oops, key not found


KeyError: 'no_key'

8. Иерархия исключений
Иерархия исключений
Исключения в Python имеют иерархию: у нас есть более общие и более специфичные исключения. Вот график исключений:

Полная иерархия 
https://docs.python.org/2/library/exceptions.html#exception-hierarchy

https://lms.skillfactory.ru/assets/courseware/v1/26c252ee13b222ccb175779fdfb154d9/asset-v1:Skillfactory+DST-PRO+15APR2020+type@asset+block/exception_hierarchy.png

Здесь мы видим, что KeyError и IndexError являются подмножеством LookupError, которая является подмножеством Exception. На практике это имеет следующее значение: если мы укажем более верхнеуровневые исключения, то "отловятся" все дочерние, но не наоборот.

In [17]:
# Напишем функцию, которая может обращаться и к спискам, и к словарям  
# при этом не выбрасывая исключение для несуществующих индексов/ключей   
def safe_element(collection, place):  
    try:  
        return(collection[place])  
    except LookupError:  
        print("Key or index not found")  
  
users = ["Pavel", "Elena", "Sergey"]  
safe_element(users, 1)  
# => 'Elena'  
safe_element(users, 3)  
# => 'Key or index not found'   
  
prices = {"apple": 10, "orange": 20}  
safe_element(prices, "apple")  
# => 10  
safe_element(prices, "carrot")  
# => 'Key or index not found'  

Key or index not found
Key or index not found


Задание 2
0.0/1.0 point (graded)
Напишите программу, которая реализует безопасное сложение двух объектов x и y.

Если объекты не могут быть сложены, функция должна:

Отловить TypeError
Вывести на экран "Can't sum x and y", где x и y - переданные числа
Вернуть 0

In [27]:
def safe_sum(x, y):
    try:        
        return x + y
    except TypeError:
        print("Can't sum x and y")
        return 0

safe_sum(5, 4)
safe_sum(5, '4')





Can't sum x and y


0

9. Детали try-except
Мы рассмотрели базовый синтаксис try-except, однако у него есть ещё несколько вариаций. Они не часто встречаются на практике, но мы о них расскажем. Иногда в блоке except нам нужен доступ к самому объекту исключения, например, мы хотим получить поясняющее сообщение и вывести его на экран, но при этом продолжить программу дальше. 
Мы можем это сделать с помощью ключевого слова 'as', за которым идёт имя новой переменной.

In [28]:
try:  
    5/0  
except ZeroDivisionError as zero_error:  
    # здесь в zero_error мы получаем сам объект исключения  
    # print как раз выведет его поясняющее сообщение  
    print(zero_error)  
  
print("Program ends correctly")  

division by zero
Program ends correctly


Ещё один распространённый подход: мы совершаем какое-то промежуточное действие, а потом перевыбрасываем исключение.

In [35]:
# Пусть у нас есть функция, которая шлёт емейл разработчику об ошибке  
def notify_admin(error):  
    print("Mail to administrator has been sent about", error)  
      
value = "poem"  
try:  
    digitized = int(value)  
except ValueError as digitized_error:  
    notify_admin(digitized_error)  
    raise digitized_error  

Mail to administrator has been sent about invalid literal for int() with base 10: 'poem'


ValueError: invalid literal for int() with base 10: 'poem'

Обратите внимание, что в начале ошибки есть уведомление о том, что email отправлен. Еще одна полезная функция: для одного try вы можете писать сразу много except на разные исключения.

In [36]:
try:  
    # открываем файл и считываем строку  
    data_file = open("valuble_data.txt")  
    s = data_file.readline()  
    # пробуем преобразовать её в число  
    i = float(s.strip())  
except OSError as err:  
    # если файла нет или его не удаётся прочитать, мы получил ошибку операционной системы   
    print("OS error: {0}".format(err))  
except ValueError:  
    # если данные не преобразуется в число, мы получим ValueError  
    print("Could not convert data to float")  

OS error: [Errno 2] No such file or directory: 'valuble_data.txt'


У try-except есть блоки else и finally; первый выполняется в случае, если мы не встретили исключение в try, и используется в основном для написания чуть более чистого кода. Finally выполняется в любом случае, даже если возникло непредвиденное исключение или выход с помощью return. Обычно используется для корректного освобождения ресурсов, например, закрытия файлов.

# Отладка: введение
В этом разделе мы обсудим подходы, которые помогают быстрее найти и исправить ошибки. Приведённые методы хорошо зарекомендовали себя на практике. Тем не менее единого метода, который бы работал всегда, нет, и разнообразие видов ошибок превращает отладку в настоящее искусство. Научиться определять необходимый метод вы сможете со временем.

Ошибки можно условно поделить на два класса:

явные, которые выбрасывают исключение;
неявные (баги): программа формально работает, но есть какой-то изъян в логике, так что вы получаете не то, что хотели.

In [37]:
import pandas as pd

In [38]:
df = pd.read_csv('imdb.csv')

In [40]:
df[:3]

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0


In [42]:
df[df['Title']==]

KeyError: 'Suicide Squad'